In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import qp
from rail.core.data import DATA_STORE
from rail.utils.catalog_utils import RomanRubinCatalogConfig
DS = DATA_STORE()

In [ ]:
from rail.raruma import utility_functions as raruma_util
from rail.raruma import plotting_functions as raruma_plot

Change this to be the root of the current PZ working area

In [ ]:
#pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'
pz_dir = '/Users/echarles/pz'

In [ ]:
d = tables_io.read(f"{pz_dir}/data/on_sky/dp1_v29.0.0_gold_all.hdf5")
pz = qp.read(f"{pz_dir}/projects/dp1/data/gold_baseline/dp1_all/output_estimate_knn.hdf5")

In [ ]:
#train = tables_io.sliceObj(d, slice(0, -1, 10))
train = d 
band_names = raruma_util.make_band_names('{band}_gaap1p0Mag', 'ugrizy')
mags = raruma_util.extract_data_to_2d_array(train,band_names)
mask = np.sum(np.isfinite(mags), axis=1) == 6
colors = raruma_util.adjacent_band_colors(mags)

In [ ]:
good_mask  = np.bitwise_and(
    np.bitwise_and(
        train['i_psfMagErr'] < 0.1,
        train['g_psfMagErr'] < 0.1,
    ),
    np.bitwise_and(
        train['r_psfMagErr'] < 0.1,
        train['z_psfMagErr'] < 0.1,
    ),
)


In [ ]:
train

In [ ]:
fluxes = raruma_util.mags_to_fluxes(mags, 31.4)

In [ ]:
fluxes = np.nan_to_num(fluxes, 200).clip(200, np.inf)

In [ ]:
pz_mode = pz.ancil['zmode']

In [ ]:
_ = raruma_plot.plot_colors_v_redshifts_with_templates(np.squeeze(pz_mode[good_mask]), colors[good_mask])

In [ ]:
_ = plt.hist(pz_mode[mask], bins=np.linspace(0., 3., 301))

In [ ]:
fluxes =  np.nan_to_num(fluxes, 200).clip(200, np.inf)
total_fluxes = fluxes[good_mask].sum(axis=1)

In [ ]:
n_obj = len(total_fluxes)

In [ ]:
good_mask.size

In [ ]:
picks = np.random.randint(n_obj, size=(n_obj))

In [ ]:
pick_fluxes = fluxes[picks]

In [ ]:
fluxes

In [ ]:
pick_totals = pick_fluxes.sum(axis=1)

In [ ]:
pick_weights = total_fluxes / pick_totals

In [ ]:
pick_weights

In [ ]:
contamination = (pick_fluxes.T*(pick_weights*0.01)).T

In [ ]:
new_fluxes = fluxes[good_mask] + contamination

In [ ]:
new_fluxes / fluxes[good_mask]

In [ ]:
(new_fluxes.sum(axis=1) / fluxes[good_mask].sum(axis=1))

In [ ]:
new_mags = np.where(np.isfinite(mags[good_mask]), raruma_util.fluxes_to_mags(new_fluxes, 31.4), np.nan)

In [ ]:
mags[good_mask] - new_mags

In [ ]:
diff_mags = mags[good_mask] - new_mags

In [ ]:
new_colors = raruma_util.adjacent_band_colors(new_mags)
colors = raruma_util.adjacent_band_colors(mags)

In [ ]:
diff_colors = new_colors - colors[good_mask]

In [ ]:
diff_colors

In [ ]:
import corner

In [ ]:
_ = corner.corner(np.nan_to_num(diff_mags.clip(-0.1, 0.1)))

In [ ]:
_ = corner.corner(np.nan_to_num(diff_colors).clip(-0.1, 0.1))

In [ ]:

_ = plt.hist(diff_colors[:,1], bins=np.linspace(-0.05, 0.05, 101))

In [ ]:
_ = plt.hist(diff_mags.std(axis=1), bins=np.linspace(-0.0, 0.05, 101))

In [ ]:
_ = plt.hist(diff_colors.std(axis=1), bins=np.linspace(0., 0.1, 101))

In [ ]:
_ = plt.hist2d(diff_colors.std(axis=1), diff_mags.mean(axis=1), bins=(np.linspace(0, 0.1, 101), np.linspace(0., 0.1, 101)), cmap='gray', norm='log')
_ = plt.xlim(0., 0.1)
_ = plt.ylim(0., 0.1)

In [ ]:
_ = plt.hist2d(diff_colors.mean(axis=1), diff_colors.std(axis=1), bins=(np.linspace(-0.1, 0.1, 101), np.linspace(0., 0.1, 101)), cmap='gray', norm='log')
_ = plt.xlim(-0.1, 0.1)
_ = plt.ylim(0., 0.1)